# Deep Convolutional GANs

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
batchSize = 64 
imageSize = 64 

transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) 

dataset = dset.CIFAR10(root = './data', download = True, transform = transform) 
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) 

/home/hvyd/anaconda3/lib/python3.5/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Files already downloaded and verified


In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Generator

In [4]:


class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output


netG = G()
netG.apply(weights_init)



G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

# Discriminator


In [5]:
class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

# Training the DCGANs

In [ ]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step() 
        
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data[0], errG.data[0]))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)
        

/home/hvyd/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0/25][0/782] Loss_D: 1.8682 Loss_G: 4.5517
[0/25][1/782] Loss_D: 1.3389 Loss_G: 6.0139
[0/25][2/782] Loss_D: 0.8093 Loss_G: 5.5869
[0/25][3/782] Loss_D: 0.9610 Loss_G: 6.1590
[0/25][4/782] Loss_D: 0.7666 Loss_G: 6.6839
[0/25][5/782] Loss_D: 1.0948 Loss_G: 5.8599
[0/25][6/782] Loss_D: 1.2627 Loss_G: 8.2935
[0/25][7/782] Loss_D: 0.6870 Loss_G: 7.9181
[0/25][8/782] Loss_D: 0.6718 Loss_G: 7.8171
[0/25][9/782] Loss_D: 0.6594 Loss_G: 9.2848
[0/25][10/782] Loss_D: 0.8783 Loss_G: 8.6805
[0/25][11/782] Loss_D: 0.9751 Loss_G: 10.2034
[0/25][12/782] Loss_D: 0.6777 Loss_G: 7.8494
[0/25][13/782] Loss_D: 1.1714 Loss_G: 12.6030
[0/25][14/782] Loss_D: 0.3321 Loss_G: 9.7251
[0/25][15/782] Loss_D: 0.7122 Loss_G: 10.3030
[0/25][16/782] Loss_D: 0.5205 Loss_G: 10.7529
[0/25][17/782] Loss_D: 0.3464 Loss_G: 9.9848
[0/25][18/782] Loss_D: 0.6510 Loss_G: 12.6782
[0/25][19/782] Loss_D: 0.2297 Loss_G: 9.3830
[0/25][20/782] Loss_D: 0.6528 Loss_G: 14.4323
[0/25][21/782] Loss_D: 0.1505 Loss_G: 12.4194
[0/25][22/782

[0/25][180/782] Loss_D: 0.2561 Loss_G: 4.9087
[0/25][181/782] Loss_D: 0.3121 Loss_G: 4.4821
[0/25][182/782] Loss_D: 0.4487 Loss_G: 5.7360
[0/25][183/782] Loss_D: 0.2215 Loss_G: 5.2288
[0/25][184/782] Loss_D: 0.2085 Loss_G: 4.8589
[0/25][185/782] Loss_D: 0.3274 Loss_G: 4.5996
[0/25][186/782] Loss_D: 0.5355 Loss_G: 8.4879
[0/25][187/782] Loss_D: 0.2839 Loss_G: 7.1639
[0/25][188/782] Loss_D: 0.1226 Loss_G: 4.6807
[0/25][189/782] Loss_D: 0.3505 Loss_G: 8.5466
[0/25][190/782] Loss_D: 0.4403 Loss_G: 5.8916
[0/25][191/782] Loss_D: 0.1758 Loss_G: 5.6044
[0/25][192/782] Loss_D: 0.2526 Loss_G: 7.1248
[0/25][193/782] Loss_D: 0.1720 Loss_G: 6.1010
[0/25][194/782] Loss_D: 0.1929 Loss_G: 5.9900
[0/25][195/782] Loss_D: 0.4476 Loss_G: 6.4023
[0/25][196/782] Loss_D: 0.2796 Loss_G: 5.6890
[0/25][197/782] Loss_D: 0.1510 Loss_G: 7.5480
[0/25][198/782] Loss_D: 0.2137 Loss_G: 5.7095
[0/25][199/782] Loss_D: 0.1728 Loss_G: 7.4634
[0/25][200/782] Loss_D: 0.1580 Loss_G: 6.2937
[0/25][201/782] Loss_D: 0.2066 Los

[0/25][358/782] Loss_D: 0.1476 Loss_G: 5.6503
[0/25][359/782] Loss_D: 0.0757 Loss_G: 6.1488
[0/25][360/782] Loss_D: 0.1568 Loss_G: 5.3165
[0/25][361/782] Loss_D: 0.0920 Loss_G: 5.2855
[0/25][362/782] Loss_D: 0.1487 Loss_G: 5.9162
[0/25][363/782] Loss_D: 0.0921 Loss_G: 5.8038
[0/25][364/782] Loss_D: 0.1671 Loss_G: 5.8410
[0/25][365/782] Loss_D: 0.1064 Loss_G: 6.3320
[0/25][366/782] Loss_D: 0.1478 Loss_G: 5.7936
[0/25][367/782] Loss_D: 0.1375 Loss_G: 6.9008
[0/25][368/782] Loss_D: 0.0545 Loss_G: 6.7630
[0/25][369/782] Loss_D: 0.1179 Loss_G: 6.0058
[0/25][370/782] Loss_D: 0.1235 Loss_G: 5.3580
[0/25][371/782] Loss_D: 0.2148 Loss_G: 6.1477
[0/25][372/782] Loss_D: 0.1089 Loss_G: 5.8225
[0/25][373/782] Loss_D: 0.0921 Loss_G: 5.6247
[0/25][374/782] Loss_D: 0.0891 Loss_G: 5.8649
[0/25][375/782] Loss_D: 0.0853 Loss_G: 5.5349
[0/25][376/782] Loss_D: 0.0926 Loss_G: 5.0685
[0/25][377/782] Loss_D: 0.0718 Loss_G: 6.1748
[0/25][378/782] Loss_D: 0.1214 Loss_G: 5.3121
[0/25][379/782] Loss_D: 0.0740 Los

[0/25][536/782] Loss_D: 0.3646 Loss_G: 9.5599
[0/25][537/782] Loss_D: 0.3914 Loss_G: 7.8024
[0/25][538/782] Loss_D: 0.2609 Loss_G: 5.2251
[0/25][539/782] Loss_D: 0.6158 Loss_G: 9.5913
[0/25][540/782] Loss_D: 1.7468 Loss_G: 3.2037
[0/25][541/782] Loss_D: 1.4679 Loss_G: 11.7525
[0/25][542/782] Loss_D: 0.4254 Loss_G: 12.7723
[0/25][543/782] Loss_D: 0.3548 Loss_G: 9.3538
[0/25][544/782] Loss_D: 0.1547 Loss_G: 6.2425
[0/25][545/782] Loss_D: 0.3235 Loss_G: 5.7829
[0/25][546/782] Loss_D: 0.2720 Loss_G: 5.1331
[0/25][547/782] Loss_D: 0.2081 Loss_G: 4.9564
[0/25][548/782] Loss_D: 0.5209 Loss_G: 6.7756
[0/25][549/782] Loss_D: 0.3490 Loss_G: 5.2879
[0/25][550/782] Loss_D: 0.2683 Loss_G: 3.6114
[0/25][551/782] Loss_D: 0.5350 Loss_G: 6.8188
[0/25][552/782] Loss_D: 0.4884 Loss_G: 5.2832
[0/25][553/782] Loss_D: 0.1724 Loss_G: 4.2460
[0/25][554/782] Loss_D: 0.2677 Loss_G: 6.2087
[0/25][555/782] Loss_D: 0.1780 Loss_G: 5.4748
[0/25][556/782] Loss_D: 0.1829 Loss_G: 3.7575
[0/25][557/782] Loss_D: 0.2278 L

[0/25][714/782] Loss_D: 0.2651 Loss_G: 3.9858
[0/25][715/782] Loss_D: 0.6669 Loss_G: 0.9295
[0/25][716/782] Loss_D: 2.2209 Loss_G: 9.6428
[0/25][717/782] Loss_D: 2.1051 Loss_G: 0.3214
[0/25][718/782] Loss_D: 2.8269 Loss_G: 7.0840
[0/25][719/782] Loss_D: 2.0729 Loss_G: 4.2516
[0/25][720/782] Loss_D: 1.4088 Loss_G: 0.9892
[0/25][721/782] Loss_D: 2.1664 Loss_G: 3.4116
[0/25][722/782] Loss_D: 1.1470 Loss_G: 2.7163
[0/25][723/782] Loss_D: 1.2076 Loss_G: 1.3325
[0/25][724/782] Loss_D: 1.4531 Loss_G: 3.1407
[0/25][725/782] Loss_D: 0.8277 Loss_G: 2.6969
[0/25][726/782] Loss_D: 1.0029 Loss_G: 2.1100
[0/25][727/782] Loss_D: 1.0526 Loss_G: 2.3997
[0/25][728/782] Loss_D: 0.8083 Loss_G: 3.2213
[0/25][729/782] Loss_D: 0.9902 Loss_G: 2.6744
[0/25][730/782] Loss_D: 0.8752 Loss_G: 2.9008
[0/25][731/782] Loss_D: 0.9428 Loss_G: 2.3822
[0/25][732/782] Loss_D: 0.9988 Loss_G: 4.2708
[0/25][733/782] Loss_D: 0.5085 Loss_G: 4.1366
[0/25][734/782] Loss_D: 0.8216 Loss_G: 3.5927
[0/25][735/782] Loss_D: 0.6757 Los

[1/25][113/782] Loss_D: 0.6362 Loss_G: 4.9584
[1/25][114/782] Loss_D: 0.4849 Loss_G: 4.1255
[1/25][115/782] Loss_D: 0.5001 Loss_G: 3.8911
[1/25][116/782] Loss_D: 0.2234 Loss_G: 5.2209
[1/25][117/782] Loss_D: 0.2188 Loss_G: 4.2716
[1/25][118/782] Loss_D: 0.1849 Loss_G: 3.7302
[1/25][119/782] Loss_D: 0.2057 Loss_G: 4.5658
[1/25][120/782] Loss_D: 0.0960 Loss_G: 5.0612
[1/25][121/782] Loss_D: 0.1108 Loss_G: 4.5526
[1/25][122/782] Loss_D: 0.2873 Loss_G: 3.4266
[1/25][123/782] Loss_D: 0.1632 Loss_G: 4.7623
[1/25][124/782] Loss_D: 0.1067 Loss_G: 4.9673
[1/25][125/782] Loss_D: 0.1982 Loss_G: 3.9500
[1/25][126/782] Loss_D: 0.1720 Loss_G: 4.6391
[1/25][127/782] Loss_D: 0.1374 Loss_G: 4.9147
[1/25][128/782] Loss_D: 0.2697 Loss_G: 3.7300
[1/25][129/782] Loss_D: 0.2159 Loss_G: 4.5003
[1/25][130/782] Loss_D: 0.1419 Loss_G: 4.6349
[1/25][131/782] Loss_D: 0.1460 Loss_G: 4.3440
[1/25][132/782] Loss_D: 0.1851 Loss_G: 4.5226
[1/25][133/782] Loss_D: 0.1903 Loss_G: 4.7067
[1/25][134/782] Loss_D: 0.1201 Los

[1/25][291/782] Loss_D: 0.5231 Loss_G: 6.6801
[1/25][292/782] Loss_D: 0.2110 Loss_G: 5.9473
[1/25][293/782] Loss_D: 0.5692 Loss_G: 2.5200
[1/25][294/782] Loss_D: 1.5799 Loss_G: 13.5685
[1/25][295/782] Loss_D: 2.4969 Loss_G: 7.1442
[1/25][296/782] Loss_D: 0.4464 Loss_G: 3.5723
[1/25][297/782] Loss_D: 1.0490 Loss_G: 9.7647
[1/25][298/782] Loss_D: 0.4735 Loss_G: 7.1376
[1/25][299/782] Loss_D: 0.6820 Loss_G: 2.4402
[1/25][300/782] Loss_D: 1.4689 Loss_G: 9.8434
[1/25][301/782] Loss_D: 0.7539 Loss_G: 7.6536
[1/25][302/782] Loss_D: 0.3938 Loss_G: 4.1587
[1/25][303/782] Loss_D: 0.5105 Loss_G: 5.9839
[1/25][304/782] Loss_D: 0.3455 Loss_G: 4.9807
[1/25][305/782] Loss_D: 0.2846 Loss_G: 4.3006
[1/25][306/782] Loss_D: 0.3792 Loss_G: 3.6340
[1/25][307/782] Loss_D: 0.3785 Loss_G: 4.5351
[1/25][308/782] Loss_D: 0.2751 Loss_G: 5.5276
[1/25][309/782] Loss_D: 0.2258 Loss_G: 4.6180
[1/25][310/782] Loss_D: 0.3669 Loss_G: 4.5688
[1/25][311/782] Loss_D: 0.6616 Loss_G: 3.5948
[1/25][312/782] Loss_D: 0.8382 Lo

[1/25][470/782] Loss_D: 0.3925 Loss_G: 4.3226
[1/25][471/782] Loss_D: 0.2030 Loss_G: 4.3816
[1/25][472/782] Loss_D: 0.3225 Loss_G: 3.6159
[1/25][473/782] Loss_D: 0.7518 Loss_G: 5.8751
[1/25][474/782] Loss_D: 1.2393 Loss_G: 1.6024
[1/25][475/782] Loss_D: 1.2841 Loss_G: 7.2688
[1/25][476/782] Loss_D: 1.0744 Loss_G: 1.7594
[1/25][477/782] Loss_D: 1.0543 Loss_G: 5.8872
[1/25][478/782] Loss_D: 1.0081 Loss_G: 3.7088
[1/25][479/782] Loss_D: 0.2290 Loss_G: 4.6937
[1/25][480/782] Loss_D: 0.3455 Loss_G: 5.0464
[1/25][481/782] Loss_D: 0.4812 Loss_G: 5.1480
[1/25][482/782] Loss_D: 0.4859 Loss_G: 3.5167
[1/25][483/782] Loss_D: 0.6227 Loss_G: 4.6635
[1/25][484/782] Loss_D: 0.5440 Loss_G: 3.4125
[1/25][485/782] Loss_D: 0.7649 Loss_G: 4.7941
[1/25][486/782] Loss_D: 0.8167 Loss_G: 2.3843
[1/25][487/782] Loss_D: 0.7773 Loss_G: 5.4751
[1/25][488/782] Loss_D: 0.9861 Loss_G: 2.4084
[1/25][489/782] Loss_D: 0.5278 Loss_G: 3.5336
[1/25][490/782] Loss_D: 0.2713 Loss_G: 4.3854
[1/25][491/782] Loss_D: 0.3176 Los

[1/25][649/782] Loss_D: 0.7936 Loss_G: 9.2326
[1/25][650/782] Loss_D: 1.4939 Loss_G: 2.9129
[1/25][651/782] Loss_D: 0.6760 Loss_G: 7.2807
[1/25][652/782] Loss_D: 0.4454 Loss_G: 3.5811
[1/25][653/782] Loss_D: 0.6109 Loss_G: 3.4742
[1/25][654/782] Loss_D: 0.7412 Loss_G: 6.9484
[1/25][655/782] Loss_D: 1.2518 Loss_G: 1.8025
[1/25][656/782] Loss_D: 1.4105 Loss_G: 7.6194
[1/25][657/782] Loss_D: 1.3022 Loss_G: 2.7797
[1/25][658/782] Loss_D: 0.6941 Loss_G: 5.3621
[1/25][659/782] Loss_D: 0.4500 Loss_G: 4.8459
[1/25][660/782] Loss_D: 0.3252 Loss_G: 3.5358
[1/25][661/782] Loss_D: 0.3668 Loss_G: 4.0123
[1/25][662/782] Loss_D: 0.5201 Loss_G: 3.7540
[1/25][663/782] Loss_D: 0.3227 Loss_G: 3.5957
[1/25][664/782] Loss_D: 0.3014 Loss_G: 3.5625
